In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
test_df = pd.read_csv ('Dataset/testset.csv')

data_test = test_df[['x_test','future']].to_numpy()

x_test_string = data_test[:,0]
y_test_string = data_test[:,1]
x_test = []
y_test = []
for i in x_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_test = [int(j) for j in i.split()]
    x_test.append(input_x_test)

for i in y_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_test = [int(j) for j in i.split()]
    y_test.append(input_y_test)
x_test = np.array(x_test)
y_test = np.array(y_test)

notes_ = notes_df.to_numpy()[:,1]
unique_notes = dict(enumerate(notes_.flatten(), 0))
# unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}

In [3]:
from MusicDataset import *
test_set = MusicDataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

In [11]:
import torch

# Specify the file path of the model to load
model_file = 'trained_model_cache/04_04_2024_00_04/model1.pth'

# Load the model
Net = torch.load(model_file, map_location='cpu')




In [8]:
total_preds = 0
correct_preds = 0
future_preds = 8
for i, data in enumerate(testloader, 0):
    input, label = data
    cumm_output = torch.zeros(0, len(unique_notes)).to(device)
    cumm_label = np.array([], dtype=int)
    for k in range(future_preds):
        # Forward pass through the model
        output = Net(input.to(device))  # Corrected call to model's forward method
        cumm_output = torch.cat((cumm_output, output))
        cumm_label = np.concatenate((cumm_label, label[:, k]))
        # Calculate accuracy
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output, 1) == label[:, k].to(device))
        input = torch.from_numpy(np.array([np.append(j, np.argmax(output.cpu().detach().numpy(), axis=1)[ind]) for ind, j in enumerate(input)])[:, 1:])
test_acc = float(correct_preds) / float(total_preds) * 100 
testreport = "Testing Accuracy:\nCorrect Predictions: {} \nTotal Predictions: {} \nTesting Accuracy: {} \n------------------------\n".format(correct_preds, total_preds, test_acc)
print(testreport)

Testing Accuracy:
Correct Predictions: 432632 
Total Predictions: 448760 
Testing Accuracy: 96.40609680007131 
------------------------



In [10]:
import os
import random
from Dataset import midi_helper

# Create bidirectionalOutputs directory if it doesn't exist
output_dir = "./stackedOutputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for j in range(10):
    index = random.randint(0,len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1,32),dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device))
        next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
                                                   for ind,j in enumerate(input)])[:,1:]) 

        tune = np.insert(tune,-1,next_preds[0])
    tune = [unique_notes[i] for i in tune]
    path = './stackedOutputs/music' + str(j) + '.midi'
    midi_helper.convert_to_midi(tune, path)